In [1]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import seaborn as sns
import sqlalchemy as sa
import matplotlib.dates as md
import matplotlib.pyplot as plt
from dotenv import dotenv_values
from datetime import datetime, timedelta
from scipy.stats import ttest_ind, chi2_contingency, mannwhitneyu

sns.set(rc={'figure.figsize':(10,5)})

In [2]:
config = dotenv_values("/home/jovyan/.env")

def get_query_clickhouse(q: str) -> pd.DataFrame:
    ch_host = config['CH_HOST']
    ch_port = config['CH_PORT']
    ch_db   = config['CH_READ_DB']
    ch_user = config['CH_READ_USER']
    ch_pass = config['CH_READ_PASS']
    
    engine = sa.create_engine(
        f"clickhouse+native://{ch_user}:"
        f"{ch_pass}@{ch_host}:"
        f"{ch_port}/{ch_db}?secure=True"
    )
    return pd.read_sql_query(q, con=engine)

In [3]:
def calculate_pvalue(test, control, metric, group_id, df, bootstrap, metric_type='proportion'):
        if metric_type == 'average' or metric_type == 'ratio':
            equal_var = False
        else:
            equal_var = True

        if metric_type == 'ratio':
            c_num_values = df[df[group_id] == control][metric[0]].values
            c_denom_values = df[df[group_id] == control][metric[1]].values
            
            t_num_values = df[df[group_id] == test][metric[0]].values
            t_denom_values = df[df[group_id] == test][metric[1]].values
                       
            control_mean = c_num_values.sum() / c_denom_values.sum()
            test_mean = t_num_values.sum() / t_denom_values.sum()

            c_values = c_num_values - c_denom_values * control_mean
            t_values = t_num_values - t_denom_values * control_mean
        else:
            c_values = df[df[group_id] == control][metric].values
            t_values = df[df[group_id] == test][metric].values

        if metric_type == 'average':
            chi_pvalue = 1
            mannwhitneyu_pvalue = 1
        else:
            chi_pvalue = chi2_contingency(np.array([[np.sum(c_values == 1), np.sum(c_values == 0)],
                                                [np.sum(t_values == 1), np.sum(t_values == 0)]]))[1]
            mannwhitneyu_pvalue = mannwhitneyu(c_values, t_values).pvalue

        if bootstrap:
            bootstrap_massive=[]
            for _ in tqdm(range(5000)):
                t_values_bootstrapped = np.concatenate((t_values, np.random.choice(t_values, size=int(bootstrap-len(t_values)), replace=True)))
                c_values_bootstrapped = np.concatenate((c_values, np.random.choice(c_values, size=int(bootstrap-len(c_values)), replace=True)))
                bootstrap_massive.append([ttest_ind(c_values_bootstrapped, t_values_bootstrapped, equal_var=equal_var).pvalue,
                                        c_values_bootstrapped.mean(), t_values_bootstrapped.mean()])
            bootstrap_massive = np.mean(np.array(bootstrap_massive), axis=0)

            print('len of groups in bootstrap:', bootstrap)
            ttest_pvalue = bootstrap_massive[0]
            control_mean = bootstrap_massive[1]
            test_mean = bootstrap_massive[2]
        else:
            ttest_pvalue = ttest_ind(c_values, t_values, equal_var=equal_var).pvalue
            control_mean = c_values.mean()
            test_mean = t_values.mean()

        print(f'control group size: {len(c_values)}, test group size: {len(t_values)}\n')
        
        return ttest_pvalue, chi_pvalue, mannwhitneyu_pvalue, control_mean, test_mean
        
def calculate_proportion_metrics(test, control, metrics, group_id, df, bootstrap=False, full_print=False):
    """
    Вычисляет значение p-value для конверсионных метрик.
    Используется стандартный t-test.
    Параметры
    ----------
    test : str
        Название тестовой группы
    control : str
        Название контрольной группы
    metrics : list of str
        Список метрик, для которых нужно посчитать статистику
    group_id : str
        Название колонки, по которой определяется группа
    df : pd.Dataframe
        Датафрейм с данными, уровень группировки - айдишник пользователя. Примеры в запросе для денежных метрик.
    bootstrap : int
        Кол-во элементов в выборке группы для проведения бутстрапа
    full_print : bool
        Булевое значение для отображеня результатов стат тестов
    Возвращает
    -------
    Печатает результат теста
    """
    for metric in metrics:
        result =  calculate_pvalue(test, control, metric, group_id, df, metric_type='proportion', bootstrap=bootstrap)
        if result[0]<0.01 or result[1]<0.01 or result[2]<0.01:
            print('STATZNACHIMO')
        elif result[0]<0.05 or result[1]<0.05 or result[2]<0.05:
            print('statznachimo?')

        if full_print:
            print('ttest p-value for', metric, ' is ', result[0], '\n',
                'chi2 p-value for', metric, ' is ', result[1], '\n',
                'mannwhitneyu p-value for', metric, ' is ', result[2], '\n',
                metric, ' mean value in control is ', result[3], '\n',
                metric, ' mean value in test is ', result[4], '\n',
                '---------------------------------------------------', '\n')
        
        print('p-value:', round(np.nanmin(result[0:3]), 2), '\n',
              '(контрольная) ', round(result[3]*100, 2), '% --> (тестовая) ', round(result[4]*100, 2), '% \n'
              'абсолютные изменения:', round(abs(result[3]-result[4])*100, 3), '% \n'
              'относительные изменения:', round((result[4] / result[3] - 1)*100, 3), '% \n'
              )
        
    return result
        
def calculate_average_metrics(test, control, metrics, group_id, df, bootstrap=False, full_print=False):
    """
    Вычисляет значение p-value для денежных и других средних поюзерных метрик.
    Используется Welch t-test с поправкой на неравенство дисперсий.
    Параметры
    ----------
    test : str
        Название тестовой группы
    control : str
        Название контрольной группы
    metrics : list of str
        Список метрик, для которых нужно посчитать статистику
    group_id : str
        Название колонки, по которой определяется группа
    df : pd.Dataframe
        Датафрейм с данными, уровень группировки - айдишник пользователя. Примеры в запросе для денежных метрик.
    bootstrap : int
        Кол-во элементов в выборке группы для проведения бутстрапа
    full_print : bool
        Булевое значение для отображеня результатов стат тестов
    Возвращает
    -------
    Печатает результат теста
    """
    for metric in metrics:
        result =  calculate_pvalue(test, control, metric, group_id, df, metric_type='average', bootstrap=bootstrap)
        if result[0]<0.01 or result[1]<0.01 or result[2]<0.01:
            print('STATZNACHIMO')
        elif result[0]<0.05 or result[1]<0.05 or result[2]<0.05:
            print('statznachimo?')

        if full_print:
            print('ttest p-value for', metric, ' is ', result[0], '\n',
                'chi2 p-value for', metric, ' is ', result[1], '\n',
                'mannwhitneyu p-value for', metric, ' is ', result[2], '\n',
                metric, ' mean value in control is ', result[3], '\n',
                metric, ' mean value in test is ', result[4], '\n',
                '---------------------------------------------------', '\n')
        
        print('p-value:', round(np.nanmin(result[0:3]), 2), '\n',
              '(контрольная) ', round(result[3], 2), ' --> (тестовая) ', round(result[4], 2), '\n'
              'абсолютные изменения:', round(abs(result[3]-result[4]), 3), '\n'
              'относительные изменения:', round((result[4] / result[3] - 1)*100, 3), '% \n'
              )
        
    return result

def calculate_ratio_metrics(test, control, metrics, group_id, df, bootstrap=False, full_print=False):
    """
    Вычисляет значение p-value для глобальных средних метрик или ratio-метрик, например для среднего чека.
    Используется Welch t-test после линеаризации - про линеаризацию тут https://instamart.atlassian.net/wiki/spaces/ANLT/pages/edit-v2/2061107789
    Параметры
    ----------
    test : str
        Название тестовой группы
    control : str
        Название контрольной группы
    metrics : list of tuples
        Список числителей и знаменателей ratio метрики в формате [('num1', 'denom1'), ('num2', 'denom2')]
    group_id : str
        Название колонки, по которой определяется группа
    df : pd.Dataframe
        Датафрейм с данными, уровень группировки тот - по какой сущности считается ratio метрика. Например, для среднего чека уровнем группировки будет order_id. Пример запроса выгружающего такой датафрейм можно найти ниже в расчете среднего чека.
    bootstrap : int
        Кол-во элементов в выборке группы для проведения бутстрапа
    full_print : bool
        Булевое значение для отображеня результатов стат тестов
    Возвращает
    -------
    Печатает результат теста
    """
    for metric in metrics:
        result =  calculate_pvalue(test, control, metric, group_id, df, metric_type='ratio', bootstrap=bootstrap)
        if result[0]<0.01 or result[1]<0.01 or result[2]<0.01:
            print('STATZNACHIMO')
        elif result[0]<0.05 or result[1]<0.05 or result[2]<0.05:
            print('statznachimo?')
            
        if full_print:
            print('ttest p-value for', metric, ' is ', result[0], '\n',
                'chi2 p-value for', metric, ' is ', result[1], '\n',
                'mannwhitneyu p-value for', metric, ' is ', result[2], '\n',
                metric, ' mean value in control is ', result[3], '\n',
                metric, ' mean value in test is ', result[4], '\n',
                '---------------------------------------------------', '\n')
        
        print('p-value:', round(np.nanmin(result[0:3]), 2), '\n',
              '(контрольная) ', round(result[3]*100, 2), '% --> (тестовая) ', round(result[4]*100, 2), '% \n'
              'абсолютные изменения:', round(abs(result[3]-result[4])*100, 3), '% \n'
              'относительные изменения:', round((result[4] / result[3] - 1)*100, 3), '% \n'
              )
        
    return result

In [8]:
# параметры эксперимента
start_date = '2024-09-25'
end_date = '2024-10-23'
exp_id = '45024ba4-877b-49a0-8238-833a4936d057'

control = 'control'
test = 'test'

# Получим данные

In [38]:
q = f"""
with 
    toDate('{start_date}') as start_date, 
    toDate('{end_date}') as end_date,
    toString('{exp_id}') as exp_id
    
, ab_groups as (
    select 
        toString(anonymous_id) as anonymous_id,
        group,
    from cdm.ab__groups__anon
    where 1=1 
        and toDate(date_msk) between start_date and end_date
        and test_id = exp_id
    group by anonymous_id, group
)

, ret as (
select distinct retailer_id from gp_rep.rep__bi_shipment where shipped_at > start_date and shipping_category_id=4 or retailer_category_name like '%%Апте%%' or retailer_category_name like '%%апте%%'
)

, f_a as (
    select anonymous_id, session_id, min(ts) as dt, uniqExact(params['retailer_id']) as uniq_ret
    from event.new_app
    where 1=1
        and toDate(ts) between start_date and end_date
        and params['retailer_id'] global not in (select * from ret)
        and event='Pickup Store List Showed'
    group by anonymous_id, session_id
)

, s_a as (
    select anonymous_id, session_id, max(ts) as dt
    from event.new_app
    where 1=1
        and toDate(ts) between start_date and end_date
        and retailer_id global not in (select * from ret)
        and event='Product Added'
        and delivery_method='pickup'
    group by anonymous_id, session_id
)

, th_a as (
    select anonymous_id, session_id, max(ts) as dt
    from event.new_app
    where 1=1
        and toDate(ts) between start_date and end_date
        and retailer_id global not in (select * from ret)
        and event='Checkout Loaded'
        and delivery_method='pickup'
    group by anonymous_id, session_id
)

, ff_a as (
    select anonymous_id, session_id, max(ts) as dt
    from event.new_app
    where 1=1
        and toDate(ts) between start_date and end_date
        and retailer_id global not in (select * from ret)
        and event='Order Completed'
        and coalesce(params['type_delivery'], delivery_method) = 'pickup'
        and params['order_number'] global in (select order_number from gp_rep.rep__bi_shipment where shipped_at > start_date and shipping_method_kind='pickup')
    group by anonymous_id, session_id
)

select 
    group,
    f_a.anonymous_id as anonymous_id, 
    max(uniq_ret) as uniq_ret,
    uniqExact(ff_a.session_id)-1 as avg_orders,
    max(if(s_a.anonymous_id<>'' and isNotNull(s_a.session_id) and s_a.dt>f_a.dt, 1, 0)) as prod_add,
    max(if(th_a.anonymous_id<>'' and isNotNull(th_a.session_id) and th_a.dt>f_a.dt, 1, 0)) as check_load,
    max(if(ff_a.anonymous_id<>'' and isNotNull(ff_a.session_id) and ff_a.dt>f_a.dt, 1, 0)) as ord_compl
from f_a
inner join ab_groups using(anonymous_id)
left join s_a on f_a.anonymous_id=s_a.anonymous_id and f_a.session_id=s_a.session_id
left join th_a on f_a.anonymous_id=th_a.anonymous_id and f_a.session_id=th_a.session_id
left join ff_a on f_a.anonymous_id=ff_a.anonymous_id and f_a.session_id=ff_a.session_id
group by group, anonymous_id

"""

df = get_query_clickhouse(q)

## Смена ритейлера в рамках заказа самовывоза

In [18]:
# без срезов
conv = calculate_average_metrics(test, control, ['uniq_ret'], 'group', 
                                          df)

control group size: 154339, test group size: 153485

p-value: 0.24 
 (контрольная)  1.35  --> (тестовая)  1.35 
абсолютные изменения: 0.004 
относительные изменения: -0.274 % 



# Частотность заказа

In [40]:
# без срезов
conv = calculate_average_metrics(test, control, ['avg_orders'], 'group', 
                                          df[df.avg_orders>0])

control group size: 3732, test group size: 3714

p-value: 0.99 
 (контрольная)  1.88  --> (тестовая)  1.89 
абсолютные изменения: 0.001 
относительные изменения: 0.027 % 



# Сквозная из выбора магазина ритейлера в заказ

In [33]:
# без срезов
conv = calculate_proportion_metrics(test, control, ['ord_compl'], 'group', 
                                          df)

control group size: 154339, test group size: 153485

p-value: 0.94 
 (контрольная)  10.23 % --> (тестовая)  10.22 % 
абсолютные изменения: 0.009 % 
относительные изменения: -0.087 % 



# Сквозная из выбора магазина ритейлера в чекаут

In [34]:
# без срезов
conv = calculate_proportion_metrics(test, control, ['check_load'], 'group', 
                                          df)

control group size: 154339, test group size: 153485

p-value: 0.72 
 (контрольная)  18.04 % --> (тестовая)  18.09 % 
абсолютные изменения: 0.05 % 
относительные изменения: 0.275 % 



# Пошаговая из выбора магазина ритейлера  в добавление товара

In [35]:
# без срезов
conv = calculate_proportion_metrics(test, control, ['prod_add'], 'group', 
                                          df)

control group size: 154339, test group size: 153485

STATZNACHIMO
p-value: 0.0 
 (контрольная)  32.21 % --> (тестовая)  34.02 % 
абсолютные изменения: 1.812 % 
относительные изменения: 5.626 % 



## Доля отмененных заказов самовывоза

In [ ]:
q = f"""
with 
    toDate('{start_date}') as start_date, 
    toDate('{end_date}') as end_date,
    toString('{exp_id}') as exp_id
    
, ab_groups as (
    select 
        toString(anonymous_id) as anonymous_id,
        group,
    from cdm.ab__groups__anon
    where 1=1 
        and toDate(date_msk) between start_date and end_date
        and test_id = exp_id
    group by anonymous_id, group
)

, ret as (
select distinct retailer_id from gp_rep.rep__bi_shipment where shipped_at > start_date and shipping_category_id=4 or retailer_category_name like '%%Апте%%' or retailer_category_name like '%%апте%%'
)

, orders as (
select 
    order_number, if(shipment_state='canceled', 1, 0) as cncl,
    gmv_goods_net_promo as aov
from gp_rep.rep__bi_shipment where completed_at > start_date and shipping_method_kind='pickup'
)

, f_a as (
    select anonymous_id, session_id, min(ts) as dt, uniqExact(params['retailer_id']) as uniq_ret
    from event.new_app
    where 1=1
        and toDate(ts) between start_date and end_date
        and params['retailer_id'] global not in (select * from ret)
        and event='Pickup Store List Showed'
    group by anonymous_id, session_id
)

, ff_a as (
    select anonymous_id, session_id, params['order_number'] as order_number
    from event.new_app
    where 1=1
        and toDate(ts) between start_date and end_date
        and retailer_id global not in (select * from ret)
        and event='Order Completed'
        and coalesce(params['type_delivery'], delivery_method) = 'pickup'
    group by anonymous_id, session_id, order_number
)

select 
    group, cncl
from f_a
inner join ab_groups using(anonymous_id)
inner join ff_a on f_a.anonymous_id=ff_a.anonymous_id and f_a.session_id=ff_a.session_id
inner join orders on ff_a.order_number=orders.order_number
"""

first = get_query_clickhouse(q)

In [36]:
conv = calculate_proportion_metrics(test, control, ['cncl'], 'group', 
                                          first)

control group size: 35503, test group size: 35090

p-value: 0.96 
 (контрольная)  21.36 % --> (тестовая)  21.37 % 
абсолютные изменения: 0.015 % 
относительные изменения: 0.069 % 



# Средний чек

In [51]:
q = f"""
with 
    toDate('{start_date}') as start_date, 
    toDate('{end_date}') as end_date,
    toString('{exp_id}') as exp_id
    
, ab_groups as (
    select 
        toString(anonymous_id) as anonymous_id,
        group,
    from cdm.ab__groups__anon
    where 1=1 
        and toDate(date_msk) between start_date and end_date
        and test_id = exp_id
    group by anonymous_id, group
)

, ret as (
select distinct retailer_id from gp_rep.rep__bi_shipment where shipped_at > start_date and shipping_category_id=4 or retailer_category_name like '%%Апте%%' or retailer_category_name like '%%апте%%'
)

, orders as (
select 
    order_number, if(shipment_state='canceled', 1, 0) as cncl,
    gmv_goods_net_promo as aov
from gp_rep.rep__bi_shipment where completed_at > start_date and shipping_method_kind='pickup'
)

, f_a as (
    select anonymous_id, session_id, min(ts) as dt, uniqExact(params['retailer_id']) as uniq_ret
    from event.new_app
    where 1=1
        and toDate(ts) between start_date and end_date
        and params['retailer_id'] global not in (select * from ret)
        and event='Pickup Store List Showed'
    group by anonymous_id, session_id
)

, ff_a as (
    select anonymous_id, session_id, params['order_number'] as order_number
    from event.new_app
    where 1=1
        and toDate(ts) between start_date and end_date
        and retailer_id global not in (select * from ret)
        and event='Order Completed'
        and coalesce(params['type_delivery'], delivery_method) = 'pickup'
    group by anonymous_id, session_id, order_number
)

select 
    group, f_a.anonymous_id as anonymous_id, sum(aov)/count(*) as aov
from f_a
    inner join ab_groups using(anonymous_id)
    inner join ff_a on f_a.anonymous_id=ff_a.anonymous_id and f_a.session_id=ff_a.session_id
    inner join orders on ff_a.order_number=orders.order_number
where cncl=0
group by group, anonymous_id
"""

second = get_query_clickhouse(q)

In [53]:
conv = calculate_average_metrics(test, control, ['aov'], 'group', 
                                          second)

control group size: 16340, test group size: 16286

p-value: 0.51 
 (контрольная)  2682.5  --> (тестовая)  2715.41 
абсолютные изменения: 32.905 
относительные изменения: 1.227 % 

